## Building a Tiny Knowledge Graph with BERT and Graph Convolutions
This notebook is the source of the code to build and test the graph described in the article


In [ ]:
import argparse
import io
import json
import os

from google.cloud import language
import numpy
import six

The following is a simple function for cleaning and extracting sentences.   it was pulled from the web.  not sure of provenance.

In [ ]:
import google

In [ ]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

# invoking the google entity extractor
to use it you must set up a google cloud account and sign up for the language.googleapis.com service.  You will also need your google credential set  up correctly.  in writing the article and debugging this notebook,  i have spent $26.   so if you have a free google cloud account it won't cost much.



In [ ]:
language_client = language.LanguageServiceClient()

def classify(text, verbose=True):
    """Classify the input text into categories. """
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT
    )
    response = language_client.analyze_entities(request={ 'document': document })
    return response

import requests
import wikipedia
def findWikiData(itemurl):
    st = itemurl.find('wiki/')
    item = itemurl[st+5:]
    #print(item)
    s= "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles=%s&format=json"%(item)
    r = requests.get(s)
    rr =r.content.decode("utf-8")
    #print('rr=', rr)
    ans = rr.find('wikibase_item')
    answer = rr[ans+16:]
    e = answer.find('"')
    return item, answer[:e]

This function grabs the instanceOf data (P31) from wikidata.  it returns an empty string if it fails to find anything.  it is basically scraping the json to find the information.  not very elegant.

In [ ]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
def getInstanceOfId(wikiID):
    s = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&languages=en&format=json'%(wikiID)
    r = requests.get(s)
    rr =r.content.decode("utf-8")
    try:
        j = json.loads(rr)
        k = rr.find('P31')
        if k > 0:
            l = rr[k:].find('"id":')
            if l > 0:
                s = rr[k+l+5:k+l+23]
                t = s.find('}')
                id = s[1:t-1]
                x = id.find('$')
                if x < 0:
                    return id
                else:
                    return id[:x]
            else:
                return ''
        else:
            return ''
    except:
        return ''
    
    

In [ ]:
getInstanceOfId('Q678023')

This is another wikidata scrape to find the name of an object given it's wikidataID

In [ ]:
def getNameFromWikiID(wikiID):
    s = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&languages=en&format=json'%(wikiID)
    r = requests.get(s)
    if r.status_code == 429:
        time.sleep(8)
        r = requests.get(s)
    rr = r.content.decode("utf-8")
    s = rr.find('"value":')
    rrr= rr[s+8:s+50]
    t = rrr.find('}')
    return rrr[1:t-1]

In [ ]:
getNameFromWikiID("Q55814")

In [ ]:
def getDescription(wikiId):
    s = 'https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&languages=en&format=json'%(wikiId)
    r = requests.get(s)
    if r.status_code == 429:
      time.sleep(8)
      r = requests.get(s)

    rr =r.content.decode("utf-8")
    i = rr.find("descriptions")
    if i < 0:
        return ''
    else:
        rrr = rr[i:]
        #print(rrr)
        j = rrr.find('"value":')
        if j >0:
            #print(j)
            k = rrr.find("}")
            return(rrr[j+8: k])
        else:
            return ''

In [ ]:
getDescription("Q55814")

This function extracts entities from a sentence.   if it is a wikipedia entity  it pulls some wikidata.

In [ ]:
def entityExtractorBlock(sents):
    ents = []
    x = classify(sents)
    for ent in x.entities:
        #print('--->', ent.name)
        if ent.metadata['wikipedia_url'] != '' or len(str.split(ent.name))> 1:
            wikipage = ent.metadata['wikipedia_url']
            if wikipage != '':
                name, wikidataId = findWikiData(wikipage)
                ents.append((name, wikidataId, wikipage))

            else:
                wikidataId = "none"
                #print(name, wikidataId, wikipage)
                ents.append((ent.name, wikidataId, wikipage))
        
    return ents

In [ ]:
import networkx as nx

The entity extractor creates a list that consists of a list of all of the entities and the last item in the list is the sentence.   

In [ ]:
def entityExtractor(sent):
    ents = []
    newents = entityExtractorBlock(sent)
    if newents != []:
        entitem = []
        for x in newents:
            entitem.append({'entity': x})
        entitem.append({'context': sent})
        ents = ents + entitem
    #print(entitem)
    return ents


## add a new file to the graph.
because the articles are named as ArtN to add a new article you need to know the number of articles currently in the graph.  that is art_cnt.

for each sentence in the article it invokes the entity extractor.  with the list of entities it creates the entity nodes for each.   for those entities that are in wikipedia it create a list of "instanceOf" entities (lightgray in color).  it finally creates an article node.   When this is all done it adds the edges as needed. 

In [ ]:
def addFile(art_cnt, G, filename):
    items = open('../'+filename+'.txt', 'r').read().replace('\n', '')
    sents = split_into_sentences(items)
    for sent in sents:
        #print(sent)
        ents = entityExtractor(sent)
        all_nodes = []
        nodes = []
        instances = []
        ent_list = []
        for e in ents:
            if e.get('entity') != None:
                ent_list.append(e['entity'][0])
                #this is an entity for this node
                lab = e['entity'][2]
                name = e['entity'][0]
                wikiID = e['entity'][1]
                st = lab.find('wiki/')
                if st <0:
                    item = name
                    #print('Item=', item)
                    G.add_node(item, flavor='entity', url='none', wikiID = 'noID', color = 'lightgreen')
                    #print("just made ", G.node[item])
                    G.nodes[item]['instanceof'] = []
                    G.nodes[item]['subclassof'] = []
                    G.nodes[item]['description'] = ''
                    nodes.append(item)

                else:
                    item = lab[st+5:]
                    #print("item=", item)
                    #wikiID = findWikiData(lab)[1]
                    G.add_node(item, flavor='entity', url=lab, wikiID = wikiID, color = 'lightblue')
                    #print("just smade ", item, G.nodes[item])
                    instID = getInstanceOfId(wikiID)
                    if instID != '':
                        inst_list = [(instID, getNameFromWikiID(instID))]
                        #print([wikiID, (instID, getNameFromWikiID(instID))])
                    else:
                        inst_list = []
                    clas_list = []#isASubclassOf(wikiID)
                    G.nodes[item]['instanceof'] = inst_list
                    instances = instances + inst_list
                    G.nodes[item]['subclassof'] = clas_list        
                    #classes = classes + clas_list
                    G.nodes[item]['description'] = getDescription(wikiID)
                    #print(findWikiData(lab))
                    nodes.append(item)
            if e.get('context') != None:
                # this is a new article
                newart = 'Art'+str(art_cnt)
                G.add_node(newart, flavor='article', source=filename, context=e['context'], color='lightyellow')
                G.nodes[newart]['ent-list'] = ent_list
                #print(newart,G.nodes[newart])
                art_cnt = art_cnt+1
                for z in nodes:
                    G.add_edge(newart,z )
                all_nodes = all_nodes + nodes
                nodes = []
        instance_set = set(instances)
        instances = list(instance_set)
        if instances != []:
            print('instances ====', instances)
        #now make items for these instances
        for i in instances:
            G.add_node(i[1], flavor='entity', url='notknown', wikiID = i[0], color = 'lightgray')
            G.nodes[i[1]]['description'] = getDescription(i[0])
            G.nodes[i[1]]['instanceof'] = []
            G.nodes[i[1]]['subclassof'] = []
        #now add edges from the instance to the new entity
        all_nodes_set = set(all_nodes)
        all_nodes =  list(all_nodes_set)
        for n in all_nodes:
            if G.nodes[n]['instanceof'] != []:
                elist = G.nodes[n]['instanceof']
                for e in elist:
                    G.add_edge(n, e[1])
    print(art_cnt)
    return art_cnt

## build the graph from the documents.
We are skiping these steps because it is easier to load the graph from the stored graph.  saves lots of time.   If you have your own set of documents you can uncoment this run it.

The next line reads the graph file from the saved copy.  

In [ ]:
#nx.write_gpickle(G, 'phy_plus_geo_singlesent')
G = nx.read_gpickle('phy_plus_geo_singlesent')

The following will plot the graph.  do not try to plot the entire graph.  too big! 
but we will use it for subgraphs.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def showGraph(sg):
    options = {
        'node_size': 3000,
        'font_weight': 'bold',
         'edgecolors':'black',
        'width': 2,
    }
    labels = {}
    color = []
    for n in sg.nodes:
        d =sg.nodes[n]
        #print(d)
        color.append(d['color'])
        labels[n]=n
    
    #print(color)
        #print(sg.nodes[x]['url'])
        #labels[]
        
    plt.figure(3,figsize=(15,15)) 

    nx.draw(sg, labels = labels, node_color = color, **options)   

utilities to inspect graph nodes.

In [ ]:
def showArticle(graph, name):
    print(name, graph.nodes[name]['context'])
    print('all edges from this node are:')
    for x in graph.edges():
        if x[1] == name:
            print(x, graph.nodes[x[0]]['url'], graph.nodes[x[0]]['wikiID'])
            

def showEntity(graph, name):
    instances = []
    classes = []
    print(name,' wikiID=', graph.nodes[name]['wikiID'], 'wikipedia url =', graph.nodes[name]['url'])
    print(graph.nodes[name]['description'])
    inst = graph.nodes[name]['instanceof'] 
    if inst != []:
        instances = instances +inst
        ilist = [i[1] for i in inst]
        print('is an instance of ', ilist)



In [ ]:
showEntity(G, 'Cretaceous')

In [ ]:
showEntity(G, 'geological period')

The scorekey function divides the world into 7 categories.   notice relativity is in all the physics categories ... because it is.

In [ ]:
climate = set(['climate', 'climate-change'])
extinction =  set(['late-devonian', 'extinction', "climate-extinctions"])
human_extinct = set(['human-extinction'])
relativity = set(['relativity'])
blackholes = set(['relativity','blackhole','blackhole-neutron', 'gravitational-wave'])
qgrav = set(['relativity', 'quantum-gravity', 'quantum-grav3', 'string-theory'])
cosmo = set(['relativity', 'cosmology'])

def scorekey(key1, key2):
    if key1 in climate and key2 in climate:
        return 1
    if key1 in extinction and key2 in extinction:
        return 1
    if key1 in human_extinct and key2 in human_extinct:
        return 1
    if key1 in relativity and key2 in relativity:
        return 1
    if key1 in blackholes and key2 in blackholes:
        return 1
    if key1 in qgrav and key2 in qgrav:
        return 1
    if key1 in cosmo and key2 in cosmo:
        return 1
    return 0

Some book keeping

In [ ]:
instances = []
classes = []
lines = []
ent_cnt = 0
for nd in G.nodes:
    #print(G.nodes[nd])
    node = G.nodes[nd]
    if node['flavor'] == 'entity':
        ent_cnt +=1
        inst = node['instanceof']
        instances = instances + inst
        clas = node['subclassof']
        classes = classes + clas
    elif node['flavor'] == 'article':
        #print('doing art ',nd, node)
        lines.append([node['context'], nd])
instanceSet = set(instances)
instances = list(instanceSet)
classSet = set(classes)
classes = list(classSet)
print('instances =', len(instances))
print("---------------")
print('classes =', len(classes))
print('lines =', len(lines))
print('entities = ', ent_cnt)

In [ ]:
sentences = [lines[i][0] for i in range(len(lines))]

In [ ]:
keys = [lines[i][1] for i in range(len(lines))]

## Create the BERT embedding
and the matrix mar which stores the normalized imbedding vectors.

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentence_embeddings = sbert_model.encode(sentences)
import numpy as np
mar = np.zeros((len(sentences), 768))
for i in range(len(sentences)):
    x = np.linalg.norm(sentence_embeddings[i])
    mar[i] = sentence_embeddings[i]/x

In [ ]:
def find_in_keys(nodeid):
    for i in range(len(keys)):
        if nodeid == keys[i]:
            return i
    return -1


## The function find_best
this function takes a sentence and look for the k closest matches based on proximity to the BERT model vectors stored in mar.   This uses dotproduct of normalized vectors which is the same as cosine distance.

In [ ]:
def find_best(k, abstract, show=True):
    v = sbert_model.encode([abstract])[0]
    v0 = v/np.linalg.norm(v)
    norms = []
    for i in range(mar.shape[0]):
        norms.append([np.dot(v0,mar[i]), i])
    norms.sort(reverse=True)
    if show:
        print('top ',k, ' related nodes' )
    scor = 0.0
    #print(norms[0][1])
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(k):
        node = keys[norms[i][1]]
        if scorekey(G.nodes[node]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sentences[norms[i][1]])
    scor = scor/k
    return scor, norms[0:k]

## find_best3
is the same as find_best but we remove candidates that are not in the same connected component.  note: k must be less than 40.   

In [ ]:
def find_best3(k, abstract, show=True):
    v = sbert_model.encode([abstract])[0]
    v0 = v/np.linalg.norm(v)
    norms = []
    for i in range(mar.shape[0]):
        norms.append([np.dot(v0,mar[i]), i])
    norms.sort(reverse=True)
    nodename = keys[norms[0][1]]
    newlist = []
    nopath = 0
    for i in range(40):
        #print(norms[i])
        x = norms[i][1]
        try:
            path = nx.shortest_path(G, nodename, 'Art'+str(x))
            #print(path)
            newlist.append(norms[i])
        except:
            nopath += 1
    norms = newlist
    ln = len(norms)
    if show:
        print('top ',k, ' related nodes' )
        if ln < k:
            print('but only found ', ln)
    scor = 0.0
    #print(norms[0][1])
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(min(ln, k)):
        node = keys[norms[i][1]]
        if scorekey(G.nodes[node]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sentences[norms[i][1]])
    scor = scor/min(ln, k)
    return scor, min(ln, k)

In [ ]:
find_best3(5, 'Black Holes are hide behind Event horizons.')

## find_best2
the function find_best2 is more complicated.  

The function findClue extracts entities from the sentence and then looks for the closest match in all of the articles among those that have the same entities.  if there are more than 2 with the max number of matches we just give up and return [].   

To compute the convolutions we need to find all the neighbors of each node.  two nodes are neighbors if they share an entity node.   note that in the computation of findNeighbors we exclude two super nodes "albert_einstein" and "earth".   these node make too many nodes neighbors. 

ComputeMar2 is the function that computes the convolution matrix mar2.   

Function find_best_wl2 is a version of find_best that uses a article nodeID rather than a document to rank near nodes (based on mar2)  it is a utility used in find_best2

Function find_best2 looks for clues.  there are either 0, 1 or 2 clues.   if there are 0 clues we use find_best(1 ...) to give us a guess.    if there are 1 or 2 clues we use them in find_best_wl2 to complete the list.  (in the case of 2 clues we use the first clue to generate the first two replies and the second clue to get the rest.  



In [ ]:
def findClue(G, sent, show = True):
    cls = entityExtractorBlock(sent)
    clsent = [x[0] for x in cls]
    if show:
        print(clsent)
    arts = []
    for x in cls:
        item = x[0]
        for edg in G.edges():
            #print(edg)
            if edg[0] == item and G.nodes[edg[1]]['flavor'] == 'article':
                arts.append(edg[1])
    artset = set(arts)
    artnames = list(artset)
    factoredartnames = []
    for art in artnames:
        #context = G.nodes[art]['context']
        #print('context=',context)
        cls1ent = G.nodes[art]['ent-list']
        #cls1ent = [x[0] for x in cls1]
        cls1int = set(clsent).intersection(set(cls1ent))
        if cls1int != set():
            factoredartnames.append((art, len(cls1int)))
        artnames = [x[0] for x in factoredartnames]
    #if show:
    #    print('artnames=',artnames)
    dic = {}
    for x in artnames:
        dic[x] = 0
    for x in factoredartnames:
        dic[x[0]] = x[1]
    #print('DIC =',dic)
    bmax = 0
    bestmax = ''
    for x in artnames:
        if dic[x] > bmax:
            bmax = dic[x]
            #print(x)
            bestmax = x
    count_ties = 0
    tie_list = []
    for x in artnames:
        if dic[x] == bmax:
            count_ties += 1
            tie_list.append(x)
    if show:
        print(count_ties, 'nodes out of',len(artnames), 'matched the max value of ', bmax)
    if count_ties >  0.5*len(artnames) and len(artnames)> 2:
        return  []
    return tie_list
   

def findNeighbors(g, art):
    n = g.nodes[art]
    #print(n)
    features = []
    neighbors = []
    for x in g.edges:
        if x[1] == art:
            #print('found art in ',x)
            p = x[0]
            features.append(p)
            if p != 'Albert_Einstein' and p != 'Earth':
                for y in g.edges:
                    if y[0] == p:
                        if y[1] != art:
                            #print(y[1][0:3])
                            if y[1][0:3] != 'Art':
                                features.append(y[1])
                            else:
                                neighbors.append(y[1])
    return neighbors, features

mar2 = np.zeros((len(sentences), 768))
def computeMar2(G, mar, lambd):
    num_zero_neighbors = 0
    tot_neighbors = 0
    num_arts = 0
    for art in G.nodes():
        if G.nodes[art]['flavor'] == 'article':
            num_arts += 1.0
            neighbors, features =  findNeighbors(G, art)
            v = np.zeros((768))
            c = 0.0
            if len(neighbors) == 0:
                num_zero_neighbors +=1
            tot_neighbors += len(neighbors)
            if len(neighbors) > 0:
                for x in neighbors:
                    loc = find_in_keys(x)
                    v = v + mar[loc]
                    c = c+1.0
                v = v/c
                z = lambd*mar[find_in_keys(art)] + (1.0-lambd)*v
                z = z/np.linalg.norm(z)
                mar2[find_in_keys(art)] = z
            else:
                mar2[find_in_keys(art)] = mar[find_in_keys(art)]

    return num_zero_neighbors, tot_neighbors/num_arts


def find_best2(k, abstract, show=True):
    cls = findClue(G, abstract, show)
    #print('clues are =', cls)
    if cls != []:
        best = cls[0]
    else:
        _, x = find_best(1, abstract, show=False)
        best = keys[x[0][1]]
    if(len(cls)> 1):
        scora, normsa = find_best_wl2(2, cls[0], show)
        scorb, normsb = find_best_wl2(k-2, cls[1], show)
        scor = (2*scora+(k-2)*scorb)/k
        norms = normsa+normsb
    else:
        scor, norms = find_best_wl2(k, best, show)
    if show:
        print('score =', scor)
    return scor, norms


def find_best_wl2(k, nodeid, show=True):
    #print(nodeid)
    v0 = mar2[find_in_keys(nodeid)]
    norms = []
    for i in range(mar2.shape[0]):
        norms.append([np.dot(v0,mar2[i]), i])
    norms.sort(reverse=True)
    score = 0.0
    #if show:
    #    print('top ',k, ' related nodes' )
    scor = 0.0
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(k):
        node = keys[norms[i][1]]
        sent = sentences[norms[i][1]]
        #print('node = ', node)
        nodea = node
        if scorekey(G.nodes[nodea]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sent)
    scor = scor/k
    return scor, norms[0:k]
            
def showRelated(G,art):
    print( G.nodes[art]['context'])
    neighbors, features =  findNeighbors(G, art)
    n = set(neighbors)
    neighbors = list(n)
    print(neighbors)
    for x in neighbors:
        print( G.nodes[x]['context'])
        v0 = mar[find_in_keys(art)]
        v1 = mar[find_in_keys(x)]
        print(" dot prod =", np.dot(v0,v1))
    print(features)



## Compute mar2
the embedding vectors for the convolution of the BERT embedding

In [ ]:
computeMar2(G, mar, 0.8)

find_best2_5 is the same as find_best2 but we eliminate all candidates that are not connected to the top choice.   This is like find_best3 above, and k must be less than 40.

In [ ]:
def find_best2_5(k, text, show = True):
    _, norms = find_best2(40, text, show = False)
    nodename = keys[norms[0][1]]
    newlist = []
    nopath = 0
    for i in range(40):
        #print(norms[i])
        x = norms[i][1]
        try:
            path = nx.shortest_path(G, nodename, 'Art'+str(x))
            #print(path)
            newlist.append(norms[i])
        except:
            nopath += 1
    norms = newlist
    ln = len(norms)

    if show:
        print('top ',k, ' related nodes' )
    scor = 0.0
    #print(norms[0][1])
    nodename = keys[norms[0][1]]
    category =  G.nodes[nodename]['source']
    for i in range(min(k,ln)):
        node = keys[norms[i][1]]
        if scorekey(G.nodes[node]['source'],category) == 1:
            scor +=1.0
        if show:
            print(node)
            print(sentences[norms[i][1]])
            
    scor = scor/min(ln, k)
    return scor, min(ln,k)

## Examples

In [ ]:
find_best2(4, 'Black Holes are hidden by an Event horizon.')

In [ ]:
find_best(4, 'What is dark energy?')

In [ ]:
find_best2(4, 'What is dark energy?')

In [ ]:
find_best(4, 'unification of general relativity and quantum theory is  supersymmetry known as supergravity')

In [ ]:
find_best2(4, 'unification of general relativity and quantum theory is  supersymmetry known as supergravity')

In [ ]:
find_best(4, 'What is M-theory and what does it have to do with string theory and supersymmetry theory')

In [ ]:
find_best2(4, 'What is M-theory and what does it have to do with string theory and supersymmetry theory')

In [ ]:
find_best(4, 'Can a neutron star and a black hole merge')

In [ ]:
find_best2(4, 'Can a neutron star and a black hole merge')

In [ ]:
find_best(4, 'How old is the universe and how much of the universe is dark matter and how much is dark energy')

In [ ]:
find_best2(4, 'How old is the universe and how much of the universe is dark matter and how much is dark energy')

In [ ]:
find_best(4, 'what is dark energy?')

In [ ]:
find_best2(4, 'what is dark energy?')

In [ ]:
find_best(4,'mass extinctions were caused by volcanic action and astroid impact.')

In [ ]:
find_best2(4,'mass extinctions were caused by volcanic action and astroid impact.')

In [ ]:
find_best(4,'What are the names of the extinction events and what is the most recent.')

In [ ]:
find_best2(4,'What are the names of the extinction events and what is the most recent.')

In [ ]:
find_best(4, 'is there a sixth mass extinction?')

In [ ]:
find_best2(4, 'is there a sixth mass extinction?')

In [ ]:
find_best(4,'The major cause of climate change is increased'+
          ' carbon dioxide levels.')

In [ ]:
find_best2(4,'The major cause of climate change is increased'+
          ' carbon dioxide levels.')

In [ ]:
find_best(4, 'Which extinction events were caused by asteroid impacts?')

In [ ]:
find_best2(4, 'Which extinction events were caused by asteroid impacts?')

In [ ]:
find_best(4, 'light neutron star')

In [ ]:
find_best2(4, 'light neutron star')

In [ ]:
find_best(4, 'event horizon?')

In [ ]:
find_best2(4, 'event horizon?')

In [ ]:
find_best(4, "who has solved Einstein's field equations?")

In [ ]:
find_best2(4, "who has solved Einstein's field equations?")

## Measuring the "accuracy" of the find_best function
below we make three measurements of the accuracy by doing 
find_best(x) for every article in the graph.  

recall that the score is not a measure of accuracy, but a measure of how similar the 2nd, 3rd and 4th reply is to the 1st.   

find_alone give a score of 70 percent accurate.
find_alone2 is better at 75% 

testing the methods that eliminate answers that are not in the same graph connected component gives even better answers, but it measures the score based on the number it found which may be less than 4. So if it finds no other answers in the component then it gets a score of 1.  

find_alone3 is a score of 80% and the average numer of items found is 3.24  
find_alone2_5 is a score of 83% and the averege number of items found is 3.33



In [ ]:
artlist = []
for n in G.nodes():
    if G.nodes[n]['flavor'] == 'article':
        artlist.append(n)

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    doc = G.nodes[n]['context']
    s, _ = find_best(4, doc, show=False)
    scor += s
print(scor/numarts)
    

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
num_retured = 0
for n in artlist:
    doc = G.nodes[n]['context']
    #print(doc)
    #s, _ = find_best(4, doc[0], show=False)
    s,  x = find_best3(4, doc, show=False)
    scor += s
    num_retured += x
print(scor/numarts, num_retured/numarts)
    

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    doc = G.nodes[n]['context']
    s, _ = find_best2(4, doc, show=False)
    scor += s
print(scor/numarts)


In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
num_returned = 0
for n in artlist:
    doc = G.nodes[n]['context']
    s, x = find_best2_5(4, doc, show=False)
    scor += s
    num_returned +=x
print(scor/numarts, num_returned/numarts)


## more layers of convolution
computerMar3 applies another layer of convolution 

In [ ]:
def computeMar3(G, mar, lambd):
    num_zero_neighbors = 0
    tot_neighbors = 0
    num_arts = 0
    for art in G.nodes():
        if G.nodes[art]['flavor'] == 'article':
            num_arts += 1.0
            neighbors, features =  findNeighbors(G, art)
            v = np.zeros((768))
            c = 0.0
            if len(neighbors) == 0:
                num_zero_neighbors +=1
            tot_neighbors += len(neighbors)
            if len(neighbors) > 0:
                for x in neighbors:
                    loc = find_in_keys(x)
                    v = v + mar[loc]
                    c = c+1.0
                v = v/c
                z = lambd*mar[find_in_keys(art)] + (1.0-lambd)*v
                z = z/np.linalg.norm(z)
                mar3[find_in_keys(art)] = z
            else:
                mar3[find_in_keys(art)] = mar[find_in_keys(art)]

    return num_zero_neighbors, tot_neighbors/num_arts


In [ ]:
mar3 = np.zeros((len(sentences), 768))
computeMar3(G, mar2,0.8)
mar2 = mar3
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    s, _ = find_best2(4, n, show=False)
    scor += s
print(scor/numarts)


In [ ]:
mar3 = np.zeros((len(sentences), 768))
computeMar3(G, mar2,0.8)
mar2 = mar3
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    s, _ = find_best_wl2(4, n, show=False)
    scor += s
print(scor/numarts)


In [ ]:
mar3 = np.zeros((len(sentences), 768))
computeMar3(G, mar2,0.8)
mar2 = mar3
numarts = len(artlist)
print(numarts)
scor = 0
for n in artlist:
    s, _ = find_best_wl2(4, n, show=False)
    scor += s
print(scor/numarts)


### Plotting the subgraph of nodes associated with a query.
this is interesting to see.   it is also interesting to look at the "story" that 
is described by following a path through the subgraph.

In [ ]:

def remove_trailing_and(text):
    i = text.find('and')
    if i >0:
        i = text.rindex('and')
        if len(text)-i < 7:
            return text[0:i]
        else:
            return text
    else:
        return text
    
def explain(sg, entity):
    n = sg.nodes[entity]
    inst = n['instanceof']
    instancefound = False
    sent =  entity 
    #print('=======================================')
    if n['description'] != '':
        preline = entity+'is defined as ', n['description']
    else: 
        preline = ''
    if inst != []:
        instancefound = True
        sent = sent +' is a '
        cnt = 0
        for ins in inst:
            endline = ''
            if cnt < len(inst)-1:
                endline = ' and '
                cnt = cnt+1
            if (ins[1].find('organism') < 0) and (ins[1].find('first-order') < 0) :
                sent = sent+ ins[1]+ endline
            else:
                cnt = cnt-1
    sent =  remove_trailing_and(sent)      
    inst = n['subclassof']
    if inst != []:
        if instancefound == True:
            sent = sent + ' and '
        else:
            sent = sent # + ' is '
        #sent = sent + ' a subclass of '
        #for ins in inst:
        #    sent = sent+ ins[1]+' and '
   
    print(preline, remove_trailing_and(sent))

In [ ]:
def makeArtSubGraph(G, artlist, do_closure = True):
    oe = G.edges
    sgl = artlist
    # compute links from articles to entities
    sgl2 = []
    for x in oe:
        if x[0] in set(sgl):
            sgl2.append(x[1])
    sgl = sgl + sgl2
    sgl = set(sgl)
    sgl = list(sgl)
    #add links from entities and articles to other things
    sgl2 = []
    for x in oe:
        if x[1] in set(sgl):
            sgl2.append(x[0])
    sgl = sgl + sgl2
    sgl = set(sgl)
    sgl = list(sgl)
    
    if do_closure:
        sgl2 = []
        for x in oe:
            if x[0] in set(sgl):
                sgl2.append(x[1])
        sgl = sgl + sgl2
        sgl = set(sgl)
        sgl = list(sgl)

    sglab = {}
    for x in sgl:
        sglab[x] = x
    sg = nx.subgraph(G, sgl)
    return sglab, sg

In [ ]:
def buildMatchingGraph(G, text , do_closure = True):
    _, s = find_best2(5, text, show=False)
    print('s=',s)
    artlist = [keys[it[1]] for it in s]
    print(len(artlist))
    for state in artlist:
        print(state+'--', G.nodes[state]['context'])
    _, sg = makeArtSubGraph(G, artlist, do_closure)
    showGraph(sg)
    print("related entities are:")
    print('=====================================')
    for nd in sg:
        n = sg.nodes[nd]
        if n['flavor']== 'entity':
            explain(sg, nd)
            print(" ")
    return sg

In [ ]:
def explainPath(cop, X, Y):
    path = nx.shortest_path(cop, X, Y)
    print(path)
    for a in path:
        print("----------------")
        if cop.nodes[a]['flavor'] == "entity":
            print(a, cop.nodes[a]['description'])
        if cop.nodes[a]['flavor'] == 'article':
            print(a, cop.nodes[a]['context'])


In [ ]:
sg = buildMatchingGraph(G,'best-known cause of a mass extinction is an Asteroid '+
                        'impact that killed off the Dinosaurs.')

In [ ]:
explainPath(sg,  'supernova explosions', 'Art407')

In [ ]:
explainPath(sg,  'Art273', 'Yucatán_Peninsula')

In [ ]:
sg =buildMatchingGraph(G,"What is dark energy?", do_closure=False)

let's compute the average number of neighbors a node has.

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
neighbors = np.zeros(5)
for n in artlist:
    nbrs, _ = findNeighbors(G, n)
    l = len(nbrs)
    if l > 4:
        l = 4
    neighbors[l]+=1
print(neighbors)


In [ ]:
tot = 0
for i in range(5):
    tot += neighbors[i]*i
tot/numarts

compute neighbors of neighbors

In [ ]:
numarts = len(artlist)
print(numarts)
scor = 0
neighbors = np.zeros(5)
for n in artlist:
    nbrs, _ = findNeighbors(G, n)
    dubnbrs = []
    for x in nbrs:
        if x != n:
            dub, _ = findNeighbors(G, x)
            dubnbrs = dubnbrs + dub
    sdubnbrs = set(dubnbrs)
    nbrs = list(sdubnbrs)
    l = len(nbrs)
    if l > 4:
        l = 4
    neighbors[l]+=1
print(neighbors)


In [ ]:
tot = 0
for i in range(5):
    tot += neighbors[i]*i
tot/numarts

In [ ]:
140/427

one third of the nodes have no neighbors.  These nodes are not effected by the convolution operation.

now  look at the sizes of the connected components.

In [ ]:
comp = nx.connected_components(G)

In [ ]:
num_comp = 0
comp_size = np.zeros(7)
for x in comp:
    x_size =  0
    for n in x:
        if G.node[n]['flavor'] == 'article':
            x_size += 1
    if x_size > 6:
        comp_size[6] = x_size
    else:
        comp_size[x_size] += 1
print(comp_size)

In [ ]:
total_nodes = 0
for i in range(6):
    total_nodes += comp_size[i]*i
total_nodes + 304

In [ ]:
tot = (76+10*2 +3*3 +4*(2+2+304))/427

In [ ]:
tot